In [1]:
# Input to pre-processing
INPUT_IMAGE_DIR = "counting_pipes/"
INPUT_LABEL_DIR = "detection_labels/"
# Output of pre-processing
CLS_ID = 1  # Just one class, 0 may be reserved for background
OUTPUT_IMAGE_DIR = "out_image_orig/"
OUTPUT_LABEL_DIR = "out_bbox_label/"
OUTPUT_BBOXIMG_DIR = "out_bbox_image/"

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import glob
import os
import numpy as np
from PIL import Image
from shutil import copyfile

In [3]:
in_labels = glob.glob(INPUT_LABEL_DIR + "*.csv")
in_images = glob.glob(INPUT_IMAGE_DIR + "*")

In [4]:
def cv2_resize(img, max_height, max_width):
    """ Stackoveflow """
    height, width = img.shape[:2]    
    # only shrink if img is bigger than required
    if max_height < height or max_width < width:
        # get scaling factor
        scaling_factor = max_height / float(height)
        if max_width/float(width) < scaling_factor:
            scaling_factor = max_width / float(width)
        # resize image
        img = cv2.resize(img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)    
    return img

In [6]:
def convert(size, box):
    """ https://github.com/Guanghan/darknet/blob/master/scripts/convert.py
    """
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [7]:
def convert_bb_to_darknet(txt_out, img_path, df):
    im = Image.open(img_path)
    w = int(im.size[0])
    h = int(im.size[1])   
    # TODO ....
    # Vectorise this but just to copy:
    # https://github.com/Guanghan/darknet/blob/master/scripts/convert.py
    with open(txt_out, "w") as txt_outfile:
        for index, row in df.iterrows():
            b = (float(row['xmin']), float(row['xmax']),
                 float(row['ymin']), float(row['ymax']))
            bb = convert((w,h), b)
            txt_outfile.write(str(CLS_ID) + " " + " ".join([str(a) for a in bb]) + '\n')

In [8]:
def draw_bboxes(img_loc, df):
    # Open image
    img = cv2.imread(img_loc)
    # Draw boxes
    for index, row in df.iterrows():
        # Draw bbox (topleft, bottomright)
        cv2.rectangle(img,
                      (row['xmin'], row['ymax']),
                      (row['xmax'], row['ymin']),
                      (0,255,0),3)
    # Resize to max-size
    img = cv2_resize(img, max_height=800, max_width=800)
    return img

In [10]:
images_done = []
for img_file in in_images:
    #print(img_file)
    # Find associated label
    # Sometimes multiple labels (use just last)
    # FLAG: Labels must be combined into one file
    file = os.path.splitext(os.path.basename(img_file))[0]
    labels = [l for l in in_labels if l.startswith(INPUT_LABEL_DIR + file)]
    if not labels:
        print("Image {} has no label".format(file))
        continue
    
    # Load labels as dataframe
    try:
        try:
            df = pd.read_csv(labels[-1], usecols=["Label", "X", "Y", "RadiusX_Pixels"])    
        except ValueError:
            df = pd.read_csv(labels[-1], usecols=["Label", "X", "Y", "Radius_Pixels"]) 
    except Exception as err:
        print("Something else {} for {}".format(err, file))
        continue
        
    # Drop label if confidence is low so no label
    df.dropna(axis=0, how='any', inplace=True)
    del df['Label']
    
    # Bounding boxes
    x, y, r = df.values[:,0], df.values[:,1], df.values[:,2] 
    # Gen co-ords for bounding box plot
    df = pd.DataFrame({'cat':[CLS_ID]*len(df),
                       'xmin':x-r,
                       'ymax':y+r,
                       'xmax':x+r,
                       'ymin':y-r})
        
    # 1. Gen bbox labels in specific darknet format
    convert_bb_to_darknet(OUTPUT_LABEL_DIR + file + '.txt', img_file, df)
    # 2. Copy image over to new folder (unttouched)
    copyfile(src=img_file, dst=OUTPUT_IMAGE_DIR+os.path.basename(img_file))
    
    # 3. Save image with bounding boxes for reference
    img = draw_bboxes(img_file, df)
    cv2.imwrite(OUTPUT_BBOXIMG_DIR + file + '.png',img)  
    
    # Log
    images_done.append(img_file)

Image IMG_0877 has no label
Image IMG_20171012_095349 has no label
Image Pipe_KI_z has no label
Image Pipe_KI_f has no label
Image IMG_20170509_135013 has no label
Image Pipe_KI_o has no label
Image IMG_20150508_100306 has no label
Image Pipe_KI_m has no label
Image Pipe_Warri_m has no label
Image Pipe_Warri2_j has no label
Image Pipe_KI_j has no label
Image Pipe_Warri_w has no label
Image IMG_20150508_103017 has no label
Image IMG_20151120_102708 has no label
Image IMG_20151120_102740 has no label
Image Pipe_KI 2_l has no label
Image Pipe_KI_y has no label
Image Pipe_Warri2_y has no label
Image DSC00042 has no label
Image Pipe_KI 3_o has no label
Image IMG_20150527_150316 has no label
Image Pipe_KI 3_c has no label
Image IMG_20150527_145005 has no label
Something else No columns to parse from file for IMG_20150525_095955
Image Pipe_KI 2_i has no label
Image IMG_0886 has no label
Image Pipe_Warri2_w has no label
Image Pipe_Warri2_c has no label
Image IMG_7514 has no label
Image Pipe_Wa

In [11]:
len(images_done)

123